In [2]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

from JumpGaussianProcess.JumpGP_LD import JumpGP_LD

NameError: name '__file__' is not defined

In [1]:
import numpy as np

x = np.random.randn(20, 2)
y = np.random.randn(20, 1)
xt = np.random.randn(1, 2)

mu_t, sig2_t, model, h = JumpGP_LD(x, y, xt, 'CEM', False)


NameError: name 'JumpGP_LD' is not defined